# Project FaceComp Controller

Controlling the computer through facial expressions. Learns the expressions of different people by analyzing them. Trained on more than 30K sample data.

<img src = "https://cdn.pixabay.com/photo/2023/05/31/15/54/ai-generated-8031745_1280.jpg" width = "40%">

## Importing necessary modules

In [ ]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks.python import vision
import numpy as np
import matplotlib.pyplot as plt

## Implementing transfer learning

To achieve better and faster results from pre trained models

<img src = "https://cdn.pixabay.com/photo/2017/09/08/19/05/a-2729782_1280.png" width = "40%">

In [ ]:
base_options = python.BaseOptions(
    model_asset_path = "./MediapipeExperiments/face_landmarker.task"
)
face_mesh = mp.solutions.face_mesh.FaceMesh(
    max_num_faces = 1,
    static_image_mode = False
)

## Image Annotator

To marks important landmarks of face, and extract coordinates of target landmarks. All the coordinates are normalized ie: in the scale of [0, 1]. Therefore to get the exact coordinates, mutiply them by the height and width of image.

In [ ]:
def image_annotator(detection_result, image, special_pt_list):

    """
    args:
    detection_result: The important landmarks detected by pre trained model
    image: Live feed from OpenCV cam
    special_pt_list: target landmarks than need to be tracked and analyzed to detect facial expressions
    """
    
    radius = 1
    color = (0, 0, 255)
    counter = 1

    # For tracking targeted landmarks
    special_pt_coordinate = []

    # Try except block to tackle the case when model failed to detect landmarks from the face
    try:

        for landmark in detection_result.multi_face_landmarks[0].landmark:

            # If current landmark is a special one
            if counter in special_pt_list:

                # Cause the coordinates are normalized
                x = int(landmark.x*width)
                y = int(landmark.y*height)
                cv2.circle(image, (x, y), 2, (0,255,0), -1)
                counter += 1

                special_pt_coordinate.append([landmark.x, landmark.y, landmark.z])

                continue

            # If it's an ordinary one
            x = int(landmark.x*width)
            y = int(landmark.y*height)
            cv2.circle(image, (x, y), radius, color, -1)

            counter += 1

        return [image, special_pt_coordinate]
    
    except Exception as e:
        return [image, None]

## Live feed input from cam
Through OpenCV

<img src = "https://cdn.pixabay.com/photo/2013/07/13/11/26/film-158157_1280.png" width = "40%">

In [ ]:
capture = cv2.VideoCapture(0)
# Setting the output screen size
cv2.namedWindow("Video", cv2.WINDOW_NORMAL)
cv2.setWindowProperty("Video", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

## Defining special points

<img src = "https://cdn.pixabay.com/photo/2017/10/29/14/02/portrait-2899779_1280.png" width = "40%">

In all, the model outputs 468 coordinates, here's the list of indices those coordinates represent. Coordinates are of the form [x, y, z]

- Nose (10 landmarks)

- Nose tip: 1
- Bridge of the nose: 2-5
- Nostrils: 6-9
- Left eye (132 landmarks)

- Eyebrow (upper lid): 33-42
- Eyelid (upper lid): 159-182
- Eyebrow (lower lid): 43-52
- Eyelid (lower lid): 263-286
- Eye (iris): 467
- Eye (pupil): 466
- Right eye (132 landmarks)

- Eyebrow (upper lid): 55-64
- Eyelid (upper lid): 374-397
- Eyebrow (lower lid): 65-74
- Eyelid (lower lid): 475-498
- Eye (iris): 468
- Eye (pupil): 469
- Lips (20 landmarks)

- Outer lips (top): 76-95
- Outer lips (bottom): 152-171
- Inner lips (top): 96-113
- Inner lips (bottom): 172-189
- Face outline (294 landmarks)

- Jawline: 0-16
- Cheek: 195-233
- Forehead: 234-280

## Defining special points

For tracking and analyzing them

In [ ]:
# Index number of those landmarks out of 468 points
special_pt_indices = [i for i in range(374, 397)]+[i for i in range(475, 498)] + [i for i in range(159, 182)] + [i for i in range(263, 286)]
# Storing their coordinates to track them
special_pts_coordinate = []

## Processing live feed

Detecting the landmarks and programming what to do with them

<img src = "https://cdn.pixabay.com/photo/2017/01/29/22/16/cycle-2019530_1280.png" width = "40%">

In [ ]:
while True:

    success, image = capture.read()
    [height, width, channels] = image.shape
    results = face_mesh.process(image)
    annotated_image = image_annotator(results, image, special_pt_indices)

    if annotated_image[1] != None: # If model didn't fail to detect landmarks
        # Storing coordinates
        print(annotated_image[1])
        special_pts_coordinate.append(annotated_image[1])
        cv2.imshow("Video", annotated_image[0])

    else:
        cv2.imshow("Video", annotated_image[0])
        
    if cv2.waitKey(1) & 0xFF == ord('q'): # Press q keyboard button to terminate the program
        np.save("data.npy", np.array(special_pts_coordinate)) # Storing tracked landmarks in numpy array format
        capture.release()
        break

capture.release()
cv2.destroyAllWindows()